In [2]:
import pandas as pd
import os
import json
import chardet

In [3]:
# print(os.getcwd())

In [189]:
# Set the display options to show all columns
pd.set_option('display.max_columns', None)
# Set to display options to show all rows
pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

In [226]:
# Path to participant simLex JSON files
directory_path = '/Users/anachkhaidze/Documents/Research Projects/Imagery Studies/simLex_imagery/data'

simLex_og_directory_path = '/Users/anachkhaidze/Documents/Research Projects/Imagery Studies/simLex_imagery/datasets/simLex.csv'

# Path to participant questionnaire JSON files
directory_path_questionnaires = '/Users/anachkhaidze/Documents/Research Projects/Imagery Studies/simLex_imagery/questionnaires/data'

In [227]:
# Function to combine simLex JSON files and convert it to csv

def process_json_files(directory_path, output_csv):
    """Process all JSON files in the directory, convert them to pandas DataFrames, and combine into one CSV file."""
    combined_df = pd.DataFrame()  # Initialize an empty DataFrame to store combined data

    # Define the required columns
    required_columns = ["item_id", "index", "value", "word1", "word2", "pos", 
                        "simlex", "concwordone", "concwordtwo", "concq", 
                        "assocusf", "stim", "simasoc", "time"]

    # Iterate over each file in the directory
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(directory_path, file_name)
            # Load the JSON file into a DataFrame from the nested 'filedata' key
            try:
                data_frame = pd.read_json(file_path)
                # Extract the array of dictionaries associated with the 'filedata' key
                df = pd.DataFrame(data_frame['filedata'].tolist())
                # Remove rows with missing values in the specified columns: To remove the last jsPsych row with irrelevant info
                df.dropna(subset=required_columns, inplace=True)
                # Append the dataframe to the combined dataframe
                combined_df = pd.concat([combined_df, df], ignore_index=True)
            except ValueError as e:
                print(f"Error reading {file_name}: {e}")

    # Once all files are processed, write the combined dataframe to a CSV
    if not combined_df.empty:
        combined_df.to_csv(output_csv, index=False)
    else:
        print("No data to write.")

# Path to the combined CSV file
output_csv = '/Users/anachkhaidze/Documents/Research Projects/Imagery Studies/simLex_imagery/data/final_combined.csv'

# Process the JSON files
process_json_files(directory_path, output_csv)

In [228]:
# Function to combine questionnaire JSON files and convert it to csv

def process_json_files(directory_path, output_csv):
    """Process all JSON files in the directory, convert them to pandas DataFrames, and combine into one CSV file."""
    combined_df = pd.DataFrame()  # Initialize an empty DataFrame to store combined data

    # Iterate over each file in the directory
    for file_name in os.listdir(directory_path_questionnaires):
        if file_name.endswith('.json'):
            file_path = os.path.join(directory_path, file_name)
            # Load the JSON file into a DataFrame from the nested 'filedata' key
            try:
                data_frame = pd.read_json(file_path)
                # Extract the array of dictionaries associated with the 'filedata' key
                df = pd.DataFrame(data_frame['filedata'].tolist())
                # Append the dataframe to the combined dataframe
                combined_df = pd.concat([combined_df, df], ignore_index=True)
            except ValueError as e:
                print(f"Error reading {file_name}: {e}")

    # Once all files are processed, write the combined dataframe to a CSV
    if not combined_df.empty:
        combined_df.to_csv(output_csv, index=False)
    else:
        print("No data to write.")

# Path to the combined CSV file
output_csv_questionnaires = '/Users/anachkhaidze/Documents/Research Projects/Imagery Studies/simLex_imagery/questionnaires/data/final_combined_questionnaires.csv'

# Process the JSON files
process_json_files(directory_path_questionnaires, output_csv_questionnaires)

In [229]:
# Read the CSV file into a dataframe
df = pd.read_csv(output_csv)
df_questionnaires = pd.read_csv(output_csv_questionnaires)

# Display the first few rows of the dataframe to confirm it's loaded correctly
# print(df.head())

# df_questionnaires

In [230]:
# Separate demographics and questionnaires

# Create an empty DataFrame to store rows for df_demographics
df_demographics = pd.DataFrame()

# Group by 'subject' and then apply a function to get the last 10 rows of each group
grouped = df_questionnaires.groupby('subject')
for name, group in grouped:
    # Append the last 10 rows of each group to df_demographics
    df_demographics = pd.concat([df_demographics, group.tail(10)])

# Now remove these rows from df_questionnaires
# Get indices of the rows that will be moved to df_demographics
indices_to_remove = df_demographics.index

# Drop these indices from df_questionnaires
df_questionnaires = df_questionnaires.drop(indices_to_remove)

# Optional: Reset index if you want clean indices in your main DataFrame
df_questionnaires.reset_index(drop=True, inplace=True)
df_demographics.reset_index(drop=True, inplace=True)

# # Print the results to verify
# print("Demographics DataFrame (last 10 rows of each subject):")
# print(df_demographics.head())
# print("Updated Questionnaires DataFrame:")
# print(df_questionnaires.head())

In [231]:
df_questionnaires

,subject,sona_id,trial_index,time_elapsed,internal_node_id,response,item_id,item,category,questionnaire
0,sncq0mnd,47767,2,53623,0.0-2.0,3,osviq-verbal-2,My verbal abilities would make a career in lan...,verbal,osivq
1,sncq0mnd,47767,2,53623,0.0-2.0,4,osviq-spatial-1,I was very good in 3D geometry as a student.,spatial,osivq
2,sncq0mnd,47767,2,53623,0.0-2.0,3,osviq-spatial-12,I am good in playing spatial games involving c...,spatial,osivq
3,sncq0mnd,47767,2,53623,0.0-2.0,1,osviq-spatial-2,If I were asked to choose between engineering ...,spatial,osivq
4,sncq0mnd,47767,2,53623,0.0-2.0,3,osviq-object-5,I enjoy pictures with bright colours and unusu...,visual,osivq
...,...,...,...,...,...,...,...,...,...,...
40193,1l23p4f0,45240,21,949497,0.0-21.0,"Perfectly realistic, as vivid as real seeing",vviq-12,You enter the shop and go to the counter. The ...,visual,vviq
40194,1l23p4f0,45240,22,981909,0.0-22.0,"Perfectly realistic, as vivid as real seeing",vviq-13,The contours of the landscape,visual,vviq
40195,1l23p4f0,45240,22,981909,0.0-22.0,Realistic and reasonably vivid,vviq-14,The color and shape of the trees,visual,vviq
40196,1l23p4f0,45240,22,981909,0.0-22.0,Realistic and reasonably vivid,vviq-15,The color and shape of the lake,visual,vviq


In [232]:
# Convert vviq responses to numbers
def update_response(value):
    # Check each condition and update the response accordingly
    if 'No image' in value:
        return 0
    elif 'Dim' in value:
        return 1
    elif 'Moderately' in value:
        return 2
    elif 'reasonably' in value:
        return 3
    elif 'Perfectly' in value:
        return 4
    else:
        return value  # Return the original value if none of the conditions are met


# Apply the function to the 'response' column
df_questionnaires['response'] = df_questionnaires['response'].apply(update_response)

In [233]:
df_questionnaires

,subject,sona_id,trial_index,time_elapsed,internal_node_id,response,item_id,item,category,questionnaire
0,sncq0mnd,47767,2,53623,0.0-2.0,3,osviq-verbal-2,My verbal abilities would make a career in lan...,verbal,osivq
1,sncq0mnd,47767,2,53623,0.0-2.0,4,osviq-spatial-1,I was very good in 3D geometry as a student.,spatial,osivq
2,sncq0mnd,47767,2,53623,0.0-2.0,3,osviq-spatial-12,I am good in playing spatial games involving c...,spatial,osivq
3,sncq0mnd,47767,2,53623,0.0-2.0,1,osviq-spatial-2,If I were asked to choose between engineering ...,spatial,osivq
4,sncq0mnd,47767,2,53623,0.0-2.0,3,osviq-object-5,I enjoy pictures with bright colours and unusu...,visual,osivq
...,...,...,...,...,...,...,...,...,...,...
40193,1l23p4f0,45240,21,949497,0.0-21.0,4,vviq-12,You enter the shop and go to the counter. The ...,visual,vviq
40194,1l23p4f0,45240,22,981909,0.0-22.0,4,vviq-13,The contours of the landscape,visual,vviq
40195,1l23p4f0,45240,22,981909,0.0-22.0,3,vviq-14,The color and shape of the trees,visual,vviq
40196,1l23p4f0,45240,22,981909,0.0-22.0,3,vviq-15,The color and shape of the lake,visual,vviq


In [234]:
# Add 1 to each response in questionnaires as the responses are indexed from 0
# Convert 'response' column to numeric, coercing errors to NaN (if there are non-numeric strings)
df_questionnaires['response'] = pd.to_numeric(df_questionnaires['response'], errors='coerce')

# Add 1 to each numeric value in the 'response' column
df_questionnaires['response'] = df_questionnaires['response'] + 1

# Rename one of the osivq_catch trials because now I have duplicate names
# Check if the 'item' column contains the word 'Elephants'
mask = df_questionnaires['item'].str.contains('Elephants', na=False)

# Update the 'item_id' column where the mask is True
df_questionnaires.loc[mask, 'item_id'] = 'osivq-catch-1'

In [235]:
# Recode Recode irq and osivq items:
# irq-verbal-10: I rarely vocalize thoughts in my mind
# irq-manipulation-5: I find it difficult to imagine how a three-dimensional geometric figure would exactly look like when rotated
# osviq-spatial-14: I find it difficult to imagine how a three-dimensional geometric figure would exactly look like when rotated
# osviq-verbal-1: I have difficulty expressing myself in writing
# osviq-verbal-4: Essay writing is difficult for me and I do not enjoy doing it at all
# osviq-verbal-15: I sometimes have a problem expressing exactly what I want to say

# Define the item IDs to look for
item_ids = ['irq-verbal-10', 'irq-manipulation-5', 'osviq-spatial-14', 'osviq-verbal-1', 'osviq-verbal-4', 'osviq-verbal-15']

# Update the response column where item_id is in the list
df_questionnaires.loc[df_questionnaires['item_id'].isin(item_ids), 'response'] = 6 - df_questionnaires['response']

# filtered_rows = df_questionnaires[df_questionnaires['item_id'].isin(item_ids)]
# filtered_rows

In [236]:
# Remove empty (irrelevant) columns from demographics dataframe
columns_to_remove = ['trial_index', 'internal_node_id', 'item_id', 'item', 'category', 'questionnaire']
df_demographics.drop(columns_to_remove, axis=1, inplace=True)

In [237]:
# Restructure demographics dataframe

# Define the new column names based on the response sequence
column_names = [
    "previous_participation", "gender", "handedness", "vision",
    "native_speaker", "education", "neurological_profile", "age",
    "feedback_about_study", "feedback_general"
]

# Create a new DataFrame for the transformed data
transformed_df = pd.DataFrame()

# Iterate over each unique subject in the DataFrame
for subject in df_demographics['subject'].unique():
    # Filter the DataFrame for the current subject
    temp_df = df_demographics[df_demographics['subject'] == subject]
    
    # Ensure that we have exactly 10 responses; if not, handle it appropriately
    if len(temp_df) == 10:
        # Extract the 'response' column and transpose it
        subject_data = temp_df['response'].values.tolist()
        # Create a DataFrame from this list and transpose it to be a single row
        subject_df = pd.DataFrame([subject_data], columns=column_names)
        # Add the 'subject' and 'sona_id' for this row
        subject_df['subject'] = subject
        subject_df['sona_id'] = temp_df['sona_id'].iloc[0]  # Assuming all sona_id's are the same per subject
        # Append to the transformed DataFrame
        transformed_df = pd.concat([transformed_df, subject_df], ignore_index=True)
    else:
        print(f"Subject {subject} does not have exactly 10 responses. Check data.")

df_demographics = transformed_df

In [238]:
# Remove empty (irrelevant) columns from questionnaires dataframe
columns_to_remove = ['trial_index', 'sona_id', 'item', 'internal_node_id', 'time_elapsed', 'category', 'questionnaire']
df_questionnaires.drop(columns_to_remove, axis=1, inplace=True)

In [239]:
# Pivot the DataFrame
df_reformatted = df_questionnaires.pivot(index='subject', columns='item_id', values='response')

# Reset the index to convert 'subject' from an index to a column
df_reformatted.reset_index(inplace=True)

# Remove the index name to clean up the DataFrame presentation
df_reformatted.columns.name = None
df_questionnaires = df_reformatted


# # Print the first few rows to verify changes
# print(df_reformatted.head())

In [240]:
# Remove empty (irrelevant) columns from simLex dataframe
columns_to_remove = ['rt', 'stimulus', 'response', 'trial_type', 'trial_index', 'time_elapsed', 'internal_node_id']
df.drop(columns_to_remove, axis=1, inplace=True)

In [241]:
## Remove subjects who did not finish questionnaires

# Calculate number of unique subjects in questionnaires and simLex
number_of_unique_subjects_simlex = df['subject'].nunique()
number_of_unique_subjects_quest = df_questionnaires['subject'].nunique()
# Get unique subjects from both DataFrames
unique_subjects_df = set(df['subject'].unique())
unique_subjects_df_questionnaires = set(df_questionnaires['subject'].unique())

# Find subjects only in df and not in df_questionnaires
only_in_df = unique_subjects_df - unique_subjects_df_questionnaires

# Find subjects only in df_questionnaires and not in df
only_in_df_questionnaires = unique_subjects_df_questionnaires - unique_subjects_df

# Calculate the number of unique subjects that are only in one of the dataframes
num_unique_only_in_one_df = len(only_in_df) + len(only_in_df_questionnaires)

# Print results
print(f"Subjects only in df: {only_in_df}")
print(f"Subjects only in df_questionnaires: {only_in_df_questionnaires}")
print(f"Number of subjects who did not finish questionnaires: {num_unique_only_in_one_df}")


# Print the number of unique subjects
print("Number of unique subjects in simLex:", number_of_unique_subjects_simlex)
print("Number of unique subjects in questionnaires:", number_of_unique_subjects_quest)

Subjects only in df: {'yqyl7k8y', 'pmz1mgaw', 'zj12zjf5', 'ywgazc12'}
Subjects only in df_questionnaires: {'v1ds0u7h'}
Number of subjects who did not finish questionnaires: 5
Number of unique subjects in simLex: 401
Number of unique subjects in questionnaires: 398


In [242]:
# Ensure all subject variable types are the same
df['subject'] = df['subject'].astype(str)
df_questionnaires['subject'] = df_questionnaires['subject'].astype(str)
df_demographics['subject'] = df_demographics['subject'].astype(str)

# Merging df with df_questionnaires on 'subject' using an inner join to keep only matching rows from 'df' and 'df_questionnaires'
merged_df = pd.merge(df, df_questionnaires, on='subject', how='inner')

# Merging the result with df_demographics on 'subject', again using an inner join to ensure all rows have corresponding data in 'df_demographics'
final_merged_df = pd.merge(merged_df, df_demographics, on='subject', how='inner', suffixes=('', '_demo'))

# Print some info to check the results
print("Number of rows in final merged DataFrame:", final_merged_df.shape[0])
print("Number of unique subjects in final merged DataFrame:", final_merged_df['subject'].nunique())

df_final = final_merged_df

Number of rows in final merged DataFrame: 39698
Number of unique subjects in final merged DataFrame: 397


In [243]:
# final_merged_df.shape
# df.shape
# df_questionnaires.shape
# df_demographics.shape

In [244]:
df_final

,item_id,index,value,word1,word2,pos,simlex,concwordone,concwordtwo,concq,assocusf,stim,simasoc,time,subject,list,lancmotorsim,lancsensorysim,lancsensorymotorsim,glovesim,attentioncheck,irq-catch-1,irq-manipulation-1,irq-manipulation-2,irq-manipulation-3,irq-manipulation-4,irq-manipulation-5,irq-manipulation-6,irq-manipulation-7,irq-manipulation-8,irq-ortho-1,irq-ortho-2,irq-ortho-3,irq-ortho-4,irq-ortho-5,irq-ortho-6,irq-ortho-7,irq-verbal-1,irq-verbal-10,irq-verbal-11,irq-verbal-12,irq-verbal-2,irq-verbal-3,irq-verbal-4,irq-verbal-5,irq-verbal-6,irq-verbal-7,irq-verbal-8,irq-verbal-9,irq-visual-1,irq-visual-10,irq-visual-2,irq-visual-3,irq-visual-4,irq-visual-5,irq-visual-6,irq-visual-7,irq-visual-8,irq-visual-9,osivq-catch-1,osivq-catch-2,osviq-object-1,osviq-object-10,osviq-object-11,osviq-object-12,osviq-object-13,osviq-object-14,osviq-object-15,osviq-object-2,osviq-object-3,osviq-object-4,osviq-object-5,osviq-object-6,osviq-object-7,osviq-object-8,osviq-object-9,osviq-spatial-1,osviq-spatial-10,osviq-spatial-11,osviq-spatial-12,osviq-spatial-13,osviq-spatial-14,osviq-spatial-15,osviq-spatial-2,osviq-spatial-3,osviq-spatial-4,osviq-spatial-5,osviq-spatial-6,osviq-spatial-7,osviq-spatial-8,osviq-spatial-9,osviq-verbal-1,osviq-verbal-10,osviq-verbal-11,osviq-verbal-12,osviq-verbal-13,osviq-verbal-14,osviq-verbal-15,osviq-verbal-2,osviq-verbal-3,osviq-verbal-4,osviq-verbal-5,osviq-verbal-6,osviq-verbal-7,osviq-verbal-8,osviq-verbal-9,vviq-1,vviq-10,vviq-11,vviq-12,vviq-13,vviq-14,vviq-15,vviq-16,vviq-2,vviq-3,vviq-4,vviq-5,vviq-6,vviq-7,vviq-8,vviq-9,previous_participation,gender,handedness,vision,native_speaker,education,neurological_profile,age,feedback_about_study,feedback_general,sona_id
0,15,1,0,hard,simple,A,1.38,3.76,1.62,2,2.16,hard,1,1.715759e+12,1ere8woo,List 1,0.144,0.273,0.226,0.626,0,1,2,2,3,1,3,2,3,1,2,2,3,1,1,2,3,2,3,4,3,4,3,3,4,4,5,4,3,3,4,2,3,3,2,3,3,2,2,5,1,2,3,2,2,2,2,1,2,4,1,3,2,2,3,2,3,1,4,2,3,3,1,2,1,3,3,2,1,4,3,5,5,3,4,3,5,3,4,3,4,5,5,4,2,4,2,3,3,1,2,1,1,2,3,2,4,4,4,4,3,5,No,Woman,Right,Yes,No,High School,['Dyslexia'],20,trying to understand how people use visual ima...,NaN,45287
1,18,4,0,happy,mad,A,0.95,2.56,2.76,2,2.07,happy,1,1.715759e+12,1ere8woo,List 1,0.026,0.026,0.027,0.712,0,1,2,2,3,1,3,2,3,1,2,2,3,1,1,2,3,2,3,4,3,4,3,3,4,4,5,4,3,3,4,2,3,3,2,3,3,2,2,5,1,2,3,2,2,2,2,1,2,4,1,3,2,2,3,2,3,1,4,2,3,3,1,2,1,3,3,2,1,4,3,5,5,3,4,3,5,3,4,3,4,5,5,4,2,4,2,3,3,1,2,1,1,2,3,2,4,4,4,4,3,5,No,Woman,Right,Yes,No,High School,['Dyslexia'],20,trying to understand how people use visual ima...,NaN,45287
2,28,4,3,narrow,broad,A,1.18,3.04,2.45,2,1.00,narrow,1,1.715759e+12,1ere8woo,List 1,0.018,0.025,0.025,0.482,0,1,2,2,3,1,3,2,3,1,2,2,3,1,1,2,3,2,3,4,3,4,3,3,4,4,5,4,3,3,4,2,3,3,2,3,3,2,2,5,1,2,3,2,2,2,2,1,2,4,1,3,2,2,3,2,3,1,4,2,3,3,1,2,1,3,3,2,1,4,3,5,5,3,4,3,5,3,4,3,4,5,5,4,2,4,2,3,3,1,2,1,1,2,3,2,4,4,4,4,3,5,No,Woman,Right,Yes,No,High School,['Dyslexia'],20,trying to understand how people use visual ima...,NaN,45287
3,43,6,6,large,big,A,9.55,3.37,3.66,2,0.68,large,1,1.715759e+12,1ere8woo,List 1,0.026,0.009,0.019,0.413,0,1,2,2,3,1,3,2,3,1,2,2,3,1,1,2,3,2,3,4,3,4,3,3,4,4,5,4,3,3,4,2,3,3,2,3,3,2,2,5,1,2,3,2,2,2,2,1,2,4,1,3,2,2,3,2,3,1,4,2,3,3,1,2,1,3,3,2,1,4,3,5,5,3,4,3,5,3,4,3,4,5,5,4,2,4,2,3,3,1,2,1,1,2,3,2,4,4,4,4,3,5,No,Woman,Right,Yes,No,High School,['Dyslexia'],20,trying to understand how people use visual ima...,NaN,45287
4,47,7,0,strong,proud,A,3.17,3.14,2.07,2,0.42,strong,0,1.715758e+12,1ere8woo,List 1,0.248,0.165,0.254,0.686,0,1,2,2,3,1,3,2,3,1,2,2,3,1,1,2,3,2,3,4,3,4,3,3,4,4,5,4,3,3,4,2,3,3,2,3,3,2,2,5,1,2,3,2,2,2,2,1,2,4,1,3,2,2,3,2,3,1,4,2,3,3,1,2,1,3,3,2,1,4,3,5,5,3,4,3,5,3,4,3,4,5,5,4,2,4,2,3,3,1,2,1,1,2,3,2,4,4,4,4,3,5,No,Woman,Right,Yes,No,High School,['Dyslexia'],20,trying to understand how people use visual ima...,NaN,45287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [245]:
# Remove non-native speakers

# Filter rows where 'native_language' equals "No"
filtered_df = df_final[df_final['native_speaker'] == "No"]

# Count unique values in 'subject' column from the filtered DataFrame
unique_subjects = filtered_df['subject'].nunique()

# Drop the rows
df_final = df_final[df_final['native_speaker'] != "No"]

# Print the number of dropped rows and unique subjects
print(f"Number of rows dropped: {filtered_df.shape[0]}")
print(f"Number of unique subjects in dropped rows: {unique_subjects}")

Number of rows dropped: 6922
Number of unique subjects in dropped rows: 69


In [246]:
print("Number of unique subjects in final merged DataFrame:", df_final['subject'].nunique())

Number of unique subjects in final merged DataFrame: 328


In [247]:
# Create new column to track attention accuracy
df_final['attention_accuracy'] = pd.NA

# Ensure 'attentioncheck' is integer for accurate comparison
df_final['attentioncheck'] = df_final['attentioncheck'].astype(int)

# Remove 'Choose_'
df_final['stim'] = df_final['stim'].str.replace('Choose_', '')
# Try to convert 'stim' to integers if all values are numeric
if df_final['stim'].str.isnumeric().all():
    df_final['stim'] = df_final['stim'].astype(int)
else:
    # Convert 'stim' to string otherwise to ensure consistent comparison
    df_final['stim'] = df_final['stim'].astype(str)

# Ensure 'value' is converted consistently for comparison
df_final['value'] = df_final['value'].astype(str)

# Define masks
mask1 = (df_final['attentioncheck'] == 1) & (df_final['value'] != '6')
mask2 = (df_final['attentioncheck'] == 1) & (df_final['value'] == '6')
mask3 = (df_final['attentioncheck'] == 2) & (df_final['value'] != df_final['stim'])
mask4 = (df_final['attentioncheck'] == 2) & (df_final['value'] == df_final['stim'])

# Apply conditions
df_final.loc[mask1, 'attention_accuracy'] = 0
df_final.loc[mask2, 'attention_accuracy'] = 1
df_final.loc[mask3, 'attention_accuracy'] = 0
df_final.loc[mask4, 'attention_accuracy'] = 1

# Debugging: Print how many rows meet each condition
# print("Rows where condition 1 is met:", mask1.sum())
# print("Rows where condition 2 is met:", mask2.sum())
# print("Rows where condition 3 is met:", mask3.sum())
# print("Rows where condition 4 is met:", mask4.sum())

# Fill NaN values in 'attention_accuracy' with -1
df_final['attention_accuracy'].fillna(-1, inplace=True)

# # Print the dataframe to check results, focusing on rows where attentioncheck is 1 or 2
# print("Detailed check for attentioncheck == 1 or 2:")
# print(df[df['attentioncheck'].isin([1, 2])][['attentioncheck', 'value', 'stim', 'attention_accuracy']].tail(20))

/var/folders/4_/n1p8tswd39b0jr38l2pdhr4r0000gn/T/ipykernel_18438/297639501.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['attention_accuracy'] = pd.NA
/var/folders/4_/n1p8tswd39b0jr38l2pdhr4r0000gn/T/ipykernel_18438/297639501.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['attentioncheck'] = df_final['attentioncheck'].astype(int)
/var/folders/4_/n1p8tswd39b0jr38l2pdhr4r0000gn/T/ipykernel_18438/297639501.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [248]:
print("Number of unique subjects in final merged DataFrame:", df_final['subject'].nunique())

Number of unique subjects in final merged DataFrame: 328


In [249]:
# Function to determine questionnaire attention accuracy
def calculate_attention(row):
    if (row['osivq-catch-1'] != 5) or (row['osivq-catch-2'] != 1) or (row['irq-catch-1'] != 1):
        return 0
    else:
        return 1

# Apply function across rows
df_final['attention_accuracy_questionnaires'] = df_final.apply(calculate_attention, axis=1)

/var/folders/4_/n1p8tswd39b0jr38l2pdhr4r0000gn/T/ipykernel_18438/2652520497.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['attention_accuracy_questionnaires'] = df_final.apply(calculate_attention, axis=1)


In [250]:
# Filter rows where attention_accuracy_questionnaires is 0
filtered_df = df_final[df_final['attention_accuracy_questionnaires'] == 0]

# Count unique subjects with attention accuracy of 0
unique_subjects_with_accuracy_0 = filtered_df['subject'].nunique()
# Print the count of unique subjects
print(f"Number of unique subjects with questionnaire accuracy = 0: {unique_subjects_with_accuracy_0}")

Number of unique subjects with questionnaire accuracy = 0: 41


In [251]:
# Filter the DataFrame to get only rows where 'attention_accuracy' is 0
df_filtered = df_final[df_final['attention_accuracy'] == 0]

# Find the number of unique subjects in the filtered DataFrame
num_unique_subjects_with_accuracy_zero = df_filtered['subject'].nunique()

# Print the result
print("Number of unique subjects with at least one row where 'attention_accuracy' is 0:", num_unique_subjects_with_accuracy_zero)

Number of unique subjects with at least one row where 'attention_accuracy' is 0: 126


In [252]:
# Function to determine overall accuracy
def calculate_attention_overall(row):
    if (row['attention_accuracy_questionnaires'] == 0) or (row['attention_accuracy'] == 0):
        return 0
    else:
        return 1

# Apply function across rows
df_final['attention_accuracy_overall'] = df_final.apply(calculate_attention_overall, axis=1)

# Filter the DataFrame to get only rows where 'attention_accuracy' is 0
df_filtered = df_final[df_final['attention_accuracy_overall'] == 0]

# Find the number of unique subjects in the filtered DataFrame
num_unique_subjects_with_accuracy_zero = df_filtered['subject'].nunique()

# Print the result
print("Number of participants is:", df_final['subject'].nunique())
print("Number of unique subjects with at least one row where 'attention_accuracy' is 0:", num_unique_subjects_with_accuracy_zero)

Number of participants is: 328
Number of unique subjects with at least one row where 'attention_accuracy' is 0: 141


/var/folders/4_/n1p8tswd39b0jr38l2pdhr4r0000gn/T/ipykernel_18438/242664399.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['attention_accuracy_overall'] = df_final.apply(calculate_attention_overall, axis=1)


In [254]:
# Identify subjects with 'attention_accuracy_overall' equal to 0
subjects_with_zero_accuracy = df_final[df_final['attention_accuracy_overall'] == 0]['subject'].unique()

# Filter out these subjects from the DataFrame
df_final = df_final[~df_final['subject'].isin(subjects_with_zero_accuracy)]

# Optionally, print the number of unique subjects dropped
print(f"Number of unique subjects dropped: {len(subjects_with_zero_accuracy)}")

Number of unique subjects dropped: 141


In [256]:
df_final['subject'].nunique()

187

In [257]:
# Frequency of unique values in 'osivq-catch-1'
print(df_final['osivq-catch-1'].value_counts())

# Frequency of unique values in 'osivq-catch-2'
print(df_final['osivq-catch-2'].value_counts())

# Frequency of unique values in 'irq-catch-1'
print(df_final['irq-catch-1'].value_counts())

osivq-catch-1
5    18608
Name: count, dtype: int64
osivq-catch-2
1    18608
Name: count, dtype: int64
irq-catch-1
1    18608
Name: count, dtype: int64


In [258]:
df_final

,item_id,index,value,word1,word2,pos,simlex,concwordone,concwordtwo,concq,assocusf,stim,simasoc,time,subject,list,lancmotorsim,lancsensorysim,lancsensorymotorsim,glovesim,attentioncheck,irq-catch-1,irq-manipulation-1,irq-manipulation-2,irq-manipulation-3,irq-manipulation-4,irq-manipulation-5,irq-manipulation-6,irq-manipulation-7,irq-manipulation-8,irq-ortho-1,irq-ortho-2,irq-ortho-3,irq-ortho-4,irq-ortho-5,irq-ortho-6,irq-ortho-7,irq-verbal-1,irq-verbal-10,irq-verbal-11,irq-verbal-12,irq-verbal-2,irq-verbal-3,irq-verbal-4,irq-verbal-5,irq-verbal-6,irq-verbal-7,irq-verbal-8,irq-verbal-9,irq-visual-1,irq-visual-10,irq-visual-2,irq-visual-3,irq-visual-4,irq-visual-5,irq-visual-6,irq-visual-7,irq-visual-8,irq-visual-9,osivq-catch-1,osivq-catch-2,osviq-object-1,osviq-object-10,osviq-object-11,osviq-object-12,osviq-object-13,osviq-object-14,osviq-object-15,osviq-object-2,osviq-object-3,osviq-object-4,osviq-object-5,osviq-object-6,osviq-object-7,osviq-object-8,osviq-object-9,osviq-spatial-1,osviq-spatial-10,osviq-spatial-11,osviq-spatial-12,osviq-spatial-13,osviq-spatial-14,osviq-spatial-15,osviq-spatial-2,osviq-spatial-3,osviq-spatial-4,osviq-spatial-5,osviq-spatial-6,osviq-spatial-7,osviq-spatial-8,osviq-spatial-9,osviq-verbal-1,osviq-verbal-10,osviq-verbal-11,osviq-verbal-12,osviq-verbal-13,osviq-verbal-14,osviq-verbal-15,osviq-verbal-2,osviq-verbal-3,osviq-verbal-4,osviq-verbal-5,osviq-verbal-6,osviq-verbal-7,osviq-verbal-8,osviq-verbal-9,vviq-1,vviq-10,vviq-11,vviq-12,vviq-13,vviq-14,vviq-15,vviq-16,vviq-2,vviq-3,vviq-4,vviq-5,vviq-6,vviq-7,vviq-8,vviq-9,previous_participation,gender,handedness,vision,native_speaker,education,neurological_profile,age,feedback_about_study,feedback_general,sona_id,attention_accuracy,attention_accuracy_questionnaires,attention_accuracy_overall
297,17,1,6,insane,crazy,A,9.57,1.77,2.37,1,2.09,insane,1,1.715810e+12,2l4en7sy,List 3,0.045,0.010,0.024,0.492,0,1,3,3,3,3,2,5,2,5,4,5,3,5,4,5,3,5,5,5,5,3,5,5,5,4,5,5,5,5,4,5,4,4,5,5,4,5,4,5,1,5,5,5,5,5,5,5,5,5,1,5,5,5,5,5,4,3,3,5,2,3,3,1,3,2,3,5,3,5,3,3,4,3,2,5,4,1,3,2,5,3,5,3,3,2,5,4,5,4,4,4,5,4,4,3,5,3,5,5,5,5,No,Woman,Right,Yes,Yes,High School,['No'],21,Visual memory,NaN,41877,-1,1,1
298,27,0,5,happy,angry,A,1.28,2.56,2.53,1,1.15,happy,1,1.715810e+12,2l4en7sy,List 3,0.005,0.018,0.014,0.629,0,1,3,3,3,3,2,5,2,5,4,5,3,5,4,5,3,5,5,5,5,3,5,5,5,4,5,5,5,5,4,5,4,4,5,5,4,5,4,5,1,5,5,5,5,5,5,5,5,5,1,5,5,5,5,5,4,3,3,5,2,3,3,1,3,2,3,5,3,5,3,3,4,3,2,5,4,1,3,2,5,3,5,3,3,2,5,4,5,4,4,4,5,4,4,3,5,3,5,5,5,5,No,Woman,Right,Yes,Yes,High School,['No'],21,Visual memory,NaN,41877,-1,1,1
299,32,4,6,inexpensive,cheap,A,8.72,2.22,2.11,1,0.79,inexpensive,1,1.715810e+12,2l4en7sy,List 3,0.009,0.018,0.014,0.235,0,1,3,3,3,3,2,5,2,5,4,5,3,5,4,5,3,5,5,5,5,3,5,5,5,4,5,5,5,5,4,5,4,4,5,5,4,5,4,5,1,5,5,5,5,5,5,5,5,5,1,5,5,5,5,5,4,3,3,5,2,3,3,1,3,2,3,5,3,5,3,3,4,3,2,5,4,1,3,2,5,3,5,3,3,2,5,4,5,4,4,4,5,4,4,3,5,3,5,5,5,5,No,Woman,Right,Yes,Yes,High School,['No'],21,Visual memory,NaN,41877,-1,1,1
300,56,7,4,honest,guilty,A,1.18,1.66,2.36,1,0.27,honest,0,1.715810e+12,2l4en7sy,List 3,0.027,0.010,0.019,0.777,0,1,3,3,3,3,2,5,2,5,4,5,3,5,4,5,3,5,5,5,5,3,5,5,5,4,5,5,5,5,4,5,4,4,5,5,4,5,4,5,1,5,5,5,5,5,5,5,5,5,1,5,5,5,5,5,4,3,3,5,2,3,3,1,3,2,3,5,3,5,3,3,4,3,2,5,4,1,3,2,5,3,5,3,3,2,5,4,5,4,4,4,5,4,4,3,5,3,5,5,5,5,No,Woman,Right,Yes,Yes,High School,['No'],21,Visual memory,NaN,41877,-1,1,1
301,60,4,6,essential,necessary,A,8.97,1.52,1.80,1,0.21,essential,0,1.715810e+12,2l4en7sy,List 3,0.007,0.021,0.025,0.334,0,1,3,3,3,3,2,5,2,5,4,5,3,5,4,5,3,5,5,5,5,3,5,5,5,4,5,5,5,5,4,5,4,4,5,5,4,5,4,5,1,5,5,5,5,5,5,5,5,5,1,5,5,5,5,5,4,3,3,5,2,3,3,1,3,2,3,5,3,5,3,3,4,3,2,5,4,1,3,2,5,3,5,3,3,2,5,4,5,4,4,4,5,4,4,3,5,3,5,5,5,5,No,Woman,Right,Yes,Yes,High School,['No'],21,Visual memory,NaN,41877,-1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [259]:
simLex_og = pd.read_csv(simLex_og_directory_path)

In [260]:
simLex_og

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex),stimulus,stimulusID,list
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41,old / new,1,1
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67,smart / intelligent,2,1
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19,hard / difficult,3,1
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18,happy / cheerful,4,1
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93,hard / easy,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,V,2.85,2.86,2.93,2,0.00,0,0.99,join / acquire,995,2
995,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44,send / attend,996,2
996,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97,gather / attend,997,2
997,absorb,withdraw,V,2.97,3.11,3.04,2,0.00,0,1.75,absorb / withdraw,998,2


In [261]:
# map the range [0,6] to the range [0,10] to align our participant responses to the og simLex similarity scores

def map_value(x):
    # Coefficients for the linear transformation from range [0, 6] to [0, 10]
    A, B = 0, 6
    C, D = 0, 10

    # Apply the linear transformation formula
    y = C + (D - C) * (x - A) / (B - A)
    return y

# Ensure 'value' column is numeric; convert if necessary
df_final['value'] = pd.to_numeric(df_final['value'], errors='coerce')

# Apply the map_value function to the 'value' column
df_final['mapped_value'] = df_final['value'].apply(map_value)

# Check for any NaN values in 'mapped_value' (which could indicate non-numeric entries in 'value')
print(df_final['mapped_value'].isna().sum(), "non-numeric entries were found and set to NaN.")

0 non-numeric entries were found and set to NaN.


In [262]:
# Merge df_final and simLex_og on 'word1' and 'word2'
merged_df = pd.merge(df_final, simLex_og, on=['word1', 'word2'])

merged_df

,item_id,index,value,word1,word2,pos,simlex,concwordone,concwordtwo,concq,assocusf,stim,simasoc,time,subject,list_x,lancmotorsim,lancsensorysim,lancsensorymotorsim,glovesim,attentioncheck,irq-catch-1,irq-manipulation-1,irq-manipulation-2,irq-manipulation-3,irq-manipulation-4,irq-manipulation-5,irq-manipulation-6,irq-manipulation-7,irq-manipulation-8,irq-ortho-1,irq-ortho-2,irq-ortho-3,irq-ortho-4,irq-ortho-5,irq-ortho-6,irq-ortho-7,irq-verbal-1,irq-verbal-10,irq-verbal-11,irq-verbal-12,irq-verbal-2,irq-verbal-3,irq-verbal-4,irq-verbal-5,irq-verbal-6,irq-verbal-7,irq-verbal-8,irq-verbal-9,irq-visual-1,irq-visual-10,irq-visual-2,irq-visual-3,irq-visual-4,irq-visual-5,irq-visual-6,irq-visual-7,irq-visual-8,irq-visual-9,osivq-catch-1,osivq-catch-2,osviq-object-1,osviq-object-10,osviq-object-11,osviq-object-12,osviq-object-13,osviq-object-14,osviq-object-15,osviq-object-2,osviq-object-3,osviq-object-4,osviq-object-5,osviq-object-6,osviq-object-7,osviq-object-8,osviq-object-9,osviq-spatial-1,osviq-spatial-10,osviq-spatial-11,osviq-spatial-12,osviq-spatial-13,osviq-spatial-14,osviq-spatial-15,osviq-spatial-2,osviq-spatial-3,osviq-spatial-4,osviq-spatial-5,osviq-spatial-6,osviq-spatial-7,osviq-spatial-8,osviq-spatial-9,osviq-verbal-1,osviq-verbal-10,osviq-verbal-11,osviq-verbal-12,osviq-verbal-13,osviq-verbal-14,osviq-verbal-15,osviq-verbal-2,osviq-verbal-3,osviq-verbal-4,osviq-verbal-5,osviq-verbal-6,osviq-verbal-7,osviq-verbal-8,osviq-verbal-9,vviq-1,vviq-10,vviq-11,vviq-12,vviq-13,vviq-14,vviq-15,vviq-16,vviq-2,vviq-3,vviq-4,vviq-5,vviq-6,vviq-7,vviq-8,vviq-9,previous_participation,gender,handedness,vision,native_speaker,education,neurological_profile,age,feedback_about_study,feedback_general,sona_id,attention_accuracy,attention_accuracy_questionnaires,attention_accuracy_overall,mapped_value,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex),stimulus,stimulusID,list_y
0,17,1,6,insane,crazy,A,9.57,1.77,2.37,1,2.09,insane,1,1.715810e+12,2l4en7sy,List 3,0.045,0.01,0.024,0.492,0,1,3,3,3,3,2,5,2,5,4,5,3,5,4,5,3,5,5,5,5,3,5,5,5,4,5,5,5,5,4,5,4,4,5,5,4,5,4,5,1,5,5,5,5,5,5,5,5,5,1,5,5,5,5,5,4,3,3,5,2,3,3,1,3,2,3,5,3,5,3,3,4,3,2,5,4,1,3,2,5,3,5,3,3,2,5,4,5,4,4,4,5,4,4,3,5,3,5,5,5,5,No,Woman,Right,Yes,Yes,High School,['No'],21,Visual memory,NaN,41877,-1,1,1,10.000000,A,9.57,1.77,2.37,1,2.09,1,0.92,insane / crazy,17,1
1,17,6,4,insane,crazy,A,9.57,1.77,2.37,1,2.09,insane,1,1.715110e+12,ads8rfju,List 3,0.045,0.01,0.024,0.492,0,1,4,4,4,2,4,3,4,4,4,2,4,2,5,4,4,3,4,3,4,4,3,4,4,3,4,4,3,4,3,4,4,4,4,3,4,4,4,5,1,3,4,3,4,4,4,4,4,4,4,4,3,4,4,4,4,3,4,3,3,4,3,2,4,4,3,4,3,4,2,4,2,4,2,4,4,2,4,3,4,4,2,2,2,3,3,3,4,4,3,3,2,3,3,4,2,4,3,3,2,4,No,Woman,Right,Yes,Yes,Bachelor's Degree,['No'],20,very interesting!,NaN,41607,-1,1,1,6.666667,A,9.57,1.77,2.37,1,2.09,1,0.92,insane / crazy,17,1
2,17,2,4,insane,crazy,A,9.57,1.77,2.37,1,2.09,insane,1,1.713921e+12,uwf4j5mh,List 3,0.045,0.01,0.024,0.492,0,1,4,5,3,5,3,4,4,2,4,3,3,2,3,4,4,4,3,4,4,4,4,3,3,3,3,3,4,5,3,3,3,4,4,4,3,3,4,5,1,2,3,3,3,3,3,3,5,2,4,4,4,3,3,3,4,2,3,3,4,3,3,3,4,3,4,3,3,4,3,3,4,4,3,4,3,4,3,3,4,4,4,4,2,2,3,3,4,4,4,4,3,4,3,2,2,3,4,3,4,3,No,Woman,Left,Yes,Yes,Bachelor's Degree,['No'],20,unsure,NaN,47171,-1,1,1,6.666667,A,9.57,1.77,2.37,1,2.09,1,0.92,insane / crazy,17,1
3,17,0,6,insane,crazy,A,9.57,1.77,2.37,1,2.09,insane,1,1.715379e+12,cofoa7vb,List 3,0.045,0.01,0.024,0.492,0,1,3,5,4,5,4,5,3,5,1,1,1,2,3,2,4,4,5,3,5,3,5,5,4,3,5,5,4,3,3,5,4,5,5,4,5,4,3,5,1,4,4,3,4,5,3,4,4,2,4,4,5,4,4,5,3,5,4,3,2,3,2,4,1,3,2,4,1,1,2,3,2,3,1,3,4,2,2,3,3,3,2,3,5,1,5,2,3,2,2,3,3,3,3,4,5,2,4,1,3,4,No,Woman,Left,Yes,Yes,High School,['No'],19,I think this study was to see how different in...,NaN,47025,-1,1,1,10.000000,A,9.57,1.77,2.37,1,2.09,1,0.92,insane / crazy,17,1
4,17,6,6,insane,crazy,A,9.57,1.77,2.37,1,2.09,insane,1,1.714782e+12,z6dto2w2,List 3,0.045,0.01,0.024,0.492,0,1,5,4,4,5,5,4,1,5,4,2,3,3,4,3,5,5,5,4,5,5,5,5,5,4,5,4,3,3,3,5,2,3,5,4,2,3,5,5,1,4,5,5,5,5,5,4,4,3,2,5,2,5,5,3,4,5,5,3,3,3,2,3,5,2,3,5,3,4,2,5

In [263]:
# Compare similarity ratings to simLex gold standard
# Calculate correlation for each subject
def calculate_correlation(group):
    return group['mapped_value'].corr(group['SimLex999'])

# Apply the correlation calculation for each subject group
subject_correlation = merged_df.groupby('subject').apply(calculate_correlation)

# Find subjects with correlation less than 0.1
low_correlation_subjects = subject_correlation[subject_correlation < 0.1].index

# Drop these subjects from the DataFrame
filtered_df = merged_df[~merged_df['subject'].isin(low_correlation_subjects)]

# Count the number of unique subjects with low correlation
num_low_correlation_subjects = len(low_correlation_subjects)

# Print the results
print(f"Number of unique subjects with correlation less than 0.1: {num_low_correlation_subjects}")


Number of unique subjects with correlation less than 0.1: 0


In [264]:
filtered_df['subject'].nunique()

187

In [265]:
df_final=filtered_df

In [266]:
# Filter columns that contain 'vviq' in their name
vviq_columns = df_final.filter(like='vviq').columns

# Sum these columns row-wise and create a new column 'vviq_sum'
df_final['vviq_sum'] = df_final[vviq_columns].sum(axis=1)
mean_vviq_sum = df_final['vviq_sum'].mean()
std_vviq_sum = df_final['vviq_sum'].std()

# Calculate z-scores for 'vviq_sum'
df_final['vviq_z'] = (df_final['vviq_sum'] - mean_vviq_sum) / std_vviq_sum

In [267]:
# Helper function to calculate the mean for specific pattern
def calculate_means(df, pattern):
    filtered_cols = df.filter(regex=pattern)
    return filtered_cols.mean(axis=1, skipna=True)

# Calculate means based on column name patterns
df_final['osivq_verbal_mean'] = calculate_means(df_final, 'osviq-verbal')
df_final['osivq_visual_mean'] = calculate_means(df_final, 'osviq-object')
df_final['osivq_spatial_mean'] = calculate_means(df_final, 'osviq-spatial')
df_final['irq_verbal_mean'] = calculate_means(df_final, 'irq-verbal')
df_final['irq_visual_mean'] = calculate_means(df_final, 'irq-visual')

# Function to calculate z-scores
def calculate_z_scores(series):
    return (series - series.mean()) / series.std()

# Calculate z-scores for each new mean column
df_final['z_osivq_verbal_mean'] = calculate_z_scores(df_final['osivq_verbal_mean'])
df_final['z_osivq_visual_mean'] = calculate_z_scores(df_final['osivq_visual_mean'])
df_final['z_osivq_spatial_mean'] = calculate_z_scores(df_final['osivq_spatial_mean'])
df_final['z_irq_verbal_mean'] = calculate_z_scores(df_final['irq_verbal_mean'])
df_final['z_irq_visual_mean'] = calculate_z_scores(df_final['irq_visual_mean'])

In [268]:
df_final

,item_id,index,value,word1,word2,pos,simlex,concwordone,concwordtwo,concq,assocusf,stim,simasoc,time,subject,list_x,lancmotorsim,lancsensorysim,lancsensorymotorsim,glovesim,attentioncheck,irq-catch-1,irq-manipulation-1,irq-manipulation-2,irq-manipulation-3,irq-manipulation-4,irq-manipulation-5,irq-manipulation-6,irq-manipulation-7,irq-manipulation-8,irq-ortho-1,irq-ortho-2,irq-ortho-3,irq-ortho-4,irq-ortho-5,irq-ortho-6,irq-ortho-7,irq-verbal-1,irq-verbal-10,irq-verbal-11,irq-verbal-12,irq-verbal-2,irq-verbal-3,irq-verbal-4,irq-verbal-5,irq-verbal-6,irq-verbal-7,irq-verbal-8,irq-verbal-9,irq-visual-1,irq-visual-10,irq-visual-2,irq-visual-3,irq-visual-4,irq-visual-5,irq-visual-6,irq-visual-7,irq-visual-8,irq-visual-9,osivq-catch-1,osivq-catch-2,osviq-object-1,osviq-object-10,osviq-object-11,osviq-object-12,osviq-object-13,osviq-object-14,osviq-object-15,osviq-object-2,osviq-object-3,osviq-object-4,osviq-object-5,osviq-object-6,osviq-object-7,osviq-object-8,osviq-object-9,osviq-spatial-1,osviq-spatial-10,osviq-spatial-11,osviq-spatial-12,osviq-spatial-13,osviq-spatial-14,osviq-spatial-15,osviq-spatial-2,osviq-spatial-3,osviq-spatial-4,osviq-spatial-5,osviq-spatial-6,osviq-spatial-7,osviq-spatial-8,osviq-spatial-9,osviq-verbal-1,osviq-verbal-10,osviq-verbal-11,osviq-verbal-12,osviq-verbal-13,osviq-verbal-14,osviq-verbal-15,osviq-verbal-2,osviq-verbal-3,osviq-verbal-4,osviq-verbal-5,osviq-verbal-6,osviq-verbal-7,osviq-verbal-8,osviq-verbal-9,vviq-1,vviq-10,vviq-11,vviq-12,vviq-13,vviq-14,vviq-15,vviq-16,vviq-2,vviq-3,vviq-4,vviq-5,vviq-6,vviq-7,vviq-8,vviq-9,previous_participation,gender,handedness,vision,native_speaker,education,neurological_profile,age,feedback_about_study,feedback_general,sona_id,attention_accuracy,attention_accuracy_questionnaires,attention_accuracy_overall,mapped_value,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex),stimulus,stimulusID,list_y,vviq_sum,vviq_z,osivq_verbal_mean,osivq_visual_mean,osivq_spatial_mean,irq_verbal_mean,irq_visual_mean,z_osivq_verbal_mean,z_osivq_visual_mean,z_osivq_spatial_mean,z_irq_verbal_mean,z_irq_visual_mean
0,17,1,6,insane,crazy,A,9.57,1.77,2.37,1,2.09,insane,1,1.715810e+12,2l4en7sy,List 3,0.045,0.01,0.024,0.492,0,1,3,3,3,3,2,5,2,5,4,5,3,5,4,5,3,5,5,5,5,3,5,5,5,4,5,5,5,5,4,5,4,4,5,5,4,5,4,5,1,5,5,5,5,5,5,5,5,5,1,5,5,5,5,5,4,3,3,5,2,3,3,1,3,2,3,5,3,5,3,3,4,3,2,5,4,1,3,2,5,3,5,3,3,2,5,4,5,4,4,4,5,4,4,3,5,3,5,5,5,5,No,Woman,Right,Yes,Yes,High School,['No'],21,Visual memory,NaN,41877,-1,1,1,10.000000,A,9.57,1.77,2.37,1,2.09,1,0.92,insane / crazy,17,1,70,1.047673,3.200000,4.733333,3.200000,4.750000,4.5,0.372449,2.072878,1.018643,1.412421,1.286155
1,17,6,4,insane,crazy,A,9.57,1.77,2.37,1,2.09,insane,1,1.715110e+12,ads8rfju,List 3,0.045,0.01,0.024,0.492,0,1,4,4,4,2,4,3,4,4,4,2,4,2,5,4,4,3,4,3,4,4,3,4,4,3,4,4,3,4,3,4,4,4,4,3,4,4,4,5,1,3,4,3,4,4,4,4,4,4,4,4,3,4,4,4,4,3,4,3,3,4,3,2,4,4,3,4,3,4,2,4,2,4,2,4,4,2,4,3,4,4,2,2,2,3,3,3,4,4,3,3,2,3,3,4,2,4,3,3,2,4,No,Woman,Right,Yes,Yes,Bachelor's Degree,['No'],20,very interesting!,NaN,41607,-1,1,1,6.666667,A,9.57,1.77,2.37,1,2.09,1,0.92,insane / crazy,17,1,50,-0.580268,3.066667,3.800000,3.333333,3.583333,3.8,0.126472,0.683250,1.256801,-0.396833,0.238274
2,17,2,4,insane,crazy,A,9.57,1.77,2.37,1,2.09,insane,1,1.713921e+12,uwf4j5mh,List 3,0.045,0.01,0.024,0.492,0,1,4,5,3,5,3,4,4,2,4,3,3,2,3,4,4,4,3,4,4,4,4,3,3,3,3,3,4,5,3,3,3,4,4,4,3,3,4,5,1,2,3,3,3,3,3,3,5,2,4,4,4,3,3,3,4,2,3,3,4,3,3,3,4,3,4,3,3,4,3,3,4,4,3,4,3,4,3,3,4,4,4,4,2,2,3,3,4,4,4,4,3,4,3,2,2,3,4,3,4,3,No,Woman,Left,Yes,Yes,Bachelor's Degree,['No'],20,unsure,NaN,47171,-1,1,1,6.666667,A,9.57,1.77,2.37,1,2.09,1,0.92,insane / crazy,17,1,53,-0.336077,3.400000,3.200000,3.266667,3.500000,3.6,0.741414,-0.210082,1.137722,-0.526065,-0.061120
3,17,0,6,insane,crazy,A,9.57,1.77,2.37,1,2.09,insane,1,1.715379e+12,cofoa7vb,List 3,0.045,0.01,0.024,0.492,0,1,3,5,4,5,4,5,3,5,1,1,1,2,3,2,4,4,5,3,5,3,5,5,4,3,5,5,4,3,3,5,4,5,5,4,5,4,3,5,1,4,4,3,4,5,3,4,4,2,4,4,5,4,4,5,3,5,4,3,2,3,2,4,1

In [269]:
df_final = df_final[df_final['attentioncheck'] == 0]

In [270]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

# Assuming df_final is your DataFrame and is already loaded

# Full model
visual_model_full = smf.mixedlm("value ~ vviq_z * lancsensorymotorsim + vviq_z * glovesim",
                                df_final, groups=df_final["subject"]).fit()

# Reduced model 1
visual_model_reduced_1 = smf.mixedlm("value ~ vviq_z + lancsensorymotorsim + vviq_z * glovesim",
                                     df_final, groups=df_final["subject"]).fit()

# Reduced model 2
visual_model_reduced_2 = smf.mixedlm("value ~ vviq_z * lancsensorymotorsim + vviq_z + glovesim",
                                     df_final, groups=df_final["subject"]).fit()

# Print the summary of the models
print("Full Model Summary:")
print(visual_model_full.summary())
print("\nReduced Model 1 Summary:")
print(visual_model_reduced_1.summary())
print("\nReduced Model 2 Summary:")
print(visual_model_reduced_2.summary())

Full Model Summary:
                Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     value      
No. Observations:      17725       Method:                 REML       
No. Groups:            187         Scale:                  3.3438     
Min. group size:       58          Log-Likelihood:         -36090.9358
Max. group size:       99          Converged:              Yes        
Mean group size:       94.8                                           
----------------------------------------------------------------------
                           Coef.  Std.Err.    z    P>|z| [0.025 0.975]
----------------------------------------------------------------------
Intercept                   5.175    0.069  75.440 0.000  5.041  5.310
vviq_z                      0.079    0.069   1.146 0.252 -0.056  0.215
lancsensorymotorsim        -4.351    0.168 -25.944 0.000 -4.680 -4.023
vviq_z:lancsensorymotorsim -0.078    0.168  -0.461 0.644 -0.407  0.252
glo

In [271]:
# Comparing models using AIC
print("AIC Comparison:")
print(f"Full Model AIC: {visual_model_full.aic}")
print(f"Reduced Model 1 AIC: {visual_model_reduced_1.aic}")
print(f"Reduced Model 2 AIC: {visual_model_reduced_2.aic}")

AIC Comparison:
Full Model AIC: nan
Reduced Model 1 AIC: nan
Reduced Model 2 AIC: nan


In [272]:
df_final.to_csv('simLex_questionnaire_cleaned.csv')